## Hello World

In [51]:
print("hellp world")

hellp world


Install Requirements ()

In [ ]:
!pip install autogen-agentchat autogen-ext[openai] -q
!pip install groq -q

In [52]:
import os
from getpass import getpass

tokenGROQ = getpass('Enter GROQ_API_KEY here: ')
os.environ["GROQ_API_KEY"] = tokenGROQ

Enter GROQ_API_KEY here: ··········


In [54]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create the agents.
custom_model_client = OpenAIChatCompletionClient(
    model="llama-3.3-70b-versatile",
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ["GROQ_API_KEY"],
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)
assistant = AssistantAgent("assistant", model_client=custom_model_client)
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("APPROVE")

# Create the team.
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a 4-line poem about the ocean.")
await Console(stream)

---------- user ----------
Write a 4-line poem about the ocean.
---------- assistant ----------
The ocean's waves crash on the shore,
A soothing sound that I adore,
The salty scent and seaweed's sway,
A calming world at the end of the day.

TERMINATE
Enter your response: APPROVE
---------- user_proxy ----------
APPROVE


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a 4-line poem about the ocean.', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=69, completion_tokens=39), content="The ocean's waves crash on the shore,\nA soothing sound that I adore,\nThe salty scent and seaweed's sway,\nA calming world at the end of the day.\n\nTERMINATE", type='TextMessage'), UserInputRequestedEvent(source='user_proxy', models_usage=None, request_id='503c1641-bb37-4a4f-ba59-b60ebfb0ff23', content='', type='UserInputRequestedEvent'), TextMessage(source='user_proxy', models_usage=None, content='APPROVE', type='TextMessage')], stop_reason="Text 'APPROVE' mentioned")

In [53]:
!pip install arxiv -q

In [56]:
import arxiv

# Construct the default API client.
client = arxiv.Client()

# Search for the 10 most recent articles matching the keyword "quantum."
search = arxiv.Search(
  query = "Multi-agent LLM systems",
  max_results = 1,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

for result in client.results(search):
  print(f"Entry ID: {result.entry_id}")
  print(f"\nTitle: {result.title}")
  print(f"\nAuthors: {', '.join([a.name for a in result.authors])}")
  print(f"\nPublished: {result.published}")
  print(f"\nUpdated: {result.updated}")
  print(f"\nSummary: {result.summary}")
  print(f"\nComment: {result.comment}")
  print(f"\nJournal Reference: {result.journal_ref}")
  print(f"\nDOI: {result.doi}")
  print(f"\nPrimary Category: {result.primary_category}")
  print(f"\nCategories: {result.categories}")
  print(f"\nPDF URL: {result.pdf_url}\n")

Entry ID: http://arxiv.org/abs/2502.13145v1

Title: Multimodal Mamba: Decoder-only Multimodal State Space Model via Quadratic to Linear Distillation

Authors: Bencheng Liao, Hongyuan Tao, Qian Zhang, Tianheng Cheng, Yingyue Li, Haoran Yin, Wenyu Liu, Xinggang Wang

Published: 2025-02-18 18:59:57+00:00

Updated: 2025-02-18 18:59:57+00:00

Summary: Recent Multimodal Large Language Models (MLLMs) have achieved remarkable
performance but face deployment challenges due to their quadratic computational
complexity, growing Key-Value cache requirements, and reliance on separate
vision encoders. We propose mmMamba, a framework for developing
linear-complexity native multimodal state space models through progressive
distillation from existing MLLMs using moderate academic computational
resources. Our approach enables the direct conversion of trained decoder-only
MLLMs to linear-complexity architectures without requiring pre-trained
RNN-based LLM or vision encoders. We propose an seeding strategy